<a href="https://colab.research.google.com/github/Maria-Evlakhova/ClinicalRecommendations/blob/main/%D0%9A%D0%BB%D0%B8%D0%BD%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5_%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gigachain gigachain-community unstructured[all-docs] faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.7/403.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                              chunk_overlap=500)
pdf_file = 'КР359_3.pdf'
loader = UnstructuredPDFLoader(pdf_file)
splitted_data = loader.load_and_split(text_splitter)

In [10]:
len(splitted_data)

822

In [11]:
splitted_data[10]

Document(metadata={'source': 'КР359_3.pdf'}, page_content='основанных на использовании природных лечебных факторов.............................................. 85\n\n5. Профилактика\n\nи\n\nдиспансерное\n\nнаблюдение, медицинские\n\nпоказания\n\nпротивопоказания к применению методов профилактики........................................................86\n\n6. Организация оказания медицинской помощи..........................................................................90\n\n7. Дополнительная информация (в том числе факторы, влияющие на исход заболевания\n\nили состояния)................................................................................................................................... 93\n\n8. Критерии оценки качества медицинской помощи..................................................................96\n\nСписок литературы...........................................................................................................................99\n\nПриложение А1. Сост

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document

In [13]:
%%time
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(splitted_data, embedding=embedding)

CPU times: user 5min 22s, sys: 18.8 s, total: 5min 41s
Wall time: 5min 43s


In [14]:
embedding_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [15]:
from google.colab import userdata
auth = userdata.get('SBER_AUTH')

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

In [17]:

llm = GigaChat(credentials=auth,
              model='GigaChat:latest',
               verify_ssl_certs=False,
               profanity_check=False)
prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, сообщи об этом пользователю.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)

In [18]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
    )

In [19]:
retrieval_chain = create_retrieval_chain(embedding_retriever, document_chain)

In [20]:
q1 = 'Как лечить астму?'

In [21]:

resp1 = retrieval_chain.invoke(
    {'input': q1}
)

In [22]:

resp1


{'input': 'Как лечить астму?',
 'context': [Document(metadata={'source': 'КР359_3.pdf'}, page_content='6.\n\nастмы) Выполнено проведение окисгенотерапии(при сатурации менее 90%) (при обострении бронхиальной астмы при отсутствии .медицинских противопоказаний)\n\n7. Выполнено назначение лекарственных препаратов групп: адренергические средства для ингаляционного введения или адренергические средства в комбинации с кортикостероидами или другими препаратами, исключая антихолинергические средства (при отсутствии медицинских противопоказаний)\n\nДа/Нет\n\n8. Выполнено назначение лекарственных препаратов действия) групп: или (системного глюкокортикоидыов кортикостероидов (ингаляционно), кроме случаев применения адренергических с в другими глюкокортикоидамикортикостероидами препаратами, исключая кроме средства) и/или блокаторов лейкотриеновых рецепторов антагонисты (в зависимости от медицинских при медицинских противопоказаний)\n\nсредств\n\nкомбинации или\n\nантихолинергическихе\n\nпоказаний\n